In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import statsmodels.formula.api as smf
import gc

In [3]:
data=pd.read_excel("Financialdata.xlsx",index_col=0)
data.fillna(0,inplace=True)
data1=data.copy()
#计算内在价值
data1["Assets"]=data1["total_assets"]
data1["Dividend"]=data["dividend_interest_payment"]
data1["DD"]=(data1["Dividend"]>0).astype(int)
data1["Earnings"]=data1["operating_profit"]
data1["NegE"]=(data1["Earnings"]>0).astype(int)
data1["Accrual"]=data1["Earnings"]-data1["net_operate_cash_flow"]

data1["Assets_1"]=data1.groupby("code")["Assets"].shift(periods=1)
data1["Dividend_1"]=data1.groupby("code")["Dividend"].shift(periods=1)
data1["DD_1"]=data1.groupby("code")["DD"].shift(periods=1)
data1["Earnings_1"]=data1.groupby("code")["Earnings"].shift(periods=1)
data1["NegE_1"]=data1.groupby("code")["NegE"].shift(periods=1)
data1["Accrual_1"]=data1.groupby("code")["Accrual"].shift(periods=1)

data1["Assets_2"]=data1.groupby("code")["Assets"].shift(periods=2)
data1["Dividend_2"]=data1.groupby("code")["Dividend"].shift(periods=2)
data1["DD_2"]=data1.groupby("code")["DD"].shift(periods=2)
data1["Earnings_2"]=data1.groupby("code")["Earnings"].shift(periods=2)
data1["NegE_2"]=data1.groupby("code")["NegE"].shift(periods=2)
data1["Accrual_2"]=data1.groupby("code")["Accrual"].shift(periods=2)

data1["Assets_3"]=data1.groupby("code")["Assets"].shift(periods=3)
data1["Dividend_3"]=data1.groupby("code")["Dividend"].shift(periods=3)
data1["DD_3"]=data1.groupby("code")["DD"].shift(periods=3)
data1["Earnings_3"]=data1.groupby("code")["Earnings"].shift(periods=3)
data1["NegE_3"]=data1.groupby("code")["NegE"].shift(periods=3)
data1["Accrual_3"]=data1.groupby("code")["Accrual"].shift(periods=3)


fit1=smf.ols("Earnings~Assets_1+Dividend_1+DD_1+Earnings_1+NegE_1+Accrual_1",data1).fit()
fit1.summary()
coef1=fit1.params

fit2=smf.ols("Earnings~Assets_2+Dividend_2+DD_2+Earnings_2+NegE_2+Accrual_2",data1).fit()
fit2.summary()
coef2=fit2.params

fit3=smf.ols("Earnings~Assets_3+Dividend_3+DD_3+Earnings_3+NegE_3+Accrual_3",data1).fit()
fit3.summary()
coef3=fit3.params

data1["EEarnings_1"]=coef1.Intercept+data1["Assets"]*coef1.Assets_1+data1["Dividend"]*coef1.Dividend_1+data1["DD"]*coef1.DD_1+data1["Earnings"]*coef1.Earnings_1+data1["NegE"]*coef1.NegE_1+data1["Accrual"]*coef1.Accrual_1
data1["EEarnings_2"]=coef2.Intercept+data1["Assets"]*coef2.Assets_2+data1["Dividend"]*coef2.Dividend_2+data1["DD"]*coef2.DD_2+data1["Earnings"]*coef2.Earnings_2+data1["NegE"]*coef2.NegE_2+data1["Accrual"]*coef2.Accrual_2
data1["EEarnings_3"]=coef3.Intercept+data1["Assets"]*coef3.Assets_3+data1["Dividend"]*coef3.Dividend_3+data1["DD"]*coef3.DD_3+data1["Earnings"]*coef3.Earnings_3+data1["NegE"]*coef3.NegE_3+data1["Accrual"]*coef3.Accrual_3

#修改股票代码，剔除ST股票
name=pd.read_excel("name&code.xlsx",index=False)
name=name.iloc[:,[1,2]]
data1_1=pd.merge(name,data1,on="code",how="inner")
data1_1.dropna(inplace=True)
data1_1.index=range(len(data1_1))

#读入个股beta数据，计算资本成本
industry=pd.read_excel("非ST全部A股行业及β.xlsx")
industry.columns=["证券代码","证券简称","行业名称","2007-12-31","2008-12-31","2009-12-31","2010-12-31","2011-12-31","2012-12-31","2013-12-31","2014-12-31","2015-12-31","2016-12-31","2017-12-31","2018-12-31","2019-12-31"]
industry=industry.melt(id_vars=["证券代码","证券简称","行业名称"],var_name='statDate',value_name='beta')
cost_rate=pd.DataFrame(industry.dropna().groupby(['statDate',"行业名称"])["beta"].median())
cost_rate["re"]=0.015+cost_rate["beta"]*0.06
cost_rate=pd.merge(industry,cost_rate,on=['statDate',"行业名称"],how="left")
cost_rate=cost_rate[["证券代码","证券简称","行业名称","statDate","re"]]

data1_2=pd.merge(data1_1,cost_rate,left_on=["statDate","name"],right_on=["statDate","证券简称"],left_index=False,right_index=False,how="inner")
data1_2["intrinsic_value"]=data1_2["total_owner_equities"]+(data1_2["EEarnings_1"]-data1_2["re"]*data1_2["total_owner_equities"])/(1+(data1_2["re"]))+(data1_2["EEarnings_2"]-data1_2["re"]*(data1_2["total_owner_equities"]+data1_2["EEarnings_1"]))/(1+data1_2["re"])**2+(data1_2["EEarnings_3"]-data1_2["re"]*(data1_2["total_owner_equities"]+data1_2["EEarnings_1"]+data1_2["EEarnings_2"]))/((1+data1_2["re"])**2)*data1_2["re"]

#计算第二个指标：盈余质量指标
data1_2['净应收款项']=data1_2['bill_receivable']+data1_2['account_receivable']+data1_2['advance_payment']-data1_2['notes_payable']-data1_2['accounts_payable']-data1_2['advance_peceipts']
data1_2['DSRI']=(data1_2['净应收款项']/data1_2['operating_revenue'])/(data1_2.groupby("code")["净应收款项"].shift(periods=1)/data1_2.groupby("code")["operating_revenue"].shift(periods=1))
data1_2['毛利率']=(data1_2["operating_revenue"]-data1_2["operating_cost"])/data1_2["operating_revenue"]
data1_2['GMI']=data1_2.groupby("code")["毛利率"].shift(periods=1)/data1_2['毛利率']
data1_2['流固金资产比例']=(data1_2['total_current_assets']+data1_2['fixed_assets']+data1_2['total_current_assets']+data1_2['biological_assets']+data1_2['intangible_assets']+data1_2['bought_sellback_assets']+data1_2['hold_for_sale_assets']+data1_2['hold_to_maturity_investments'])/data1_2['total_assets']
data1_2['AQI']=(1-data1_2['流固金资产比例'])/(1-data1_2.groupby("code")["流固金资产比例"].shift(periods=1))
data1_2['SGI']=data1_2['operating_revenue']/data1_2.groupby("code")["operating_revenue"].shift(periods=1)
data1_2['折旧摊销']=data1_2['fixed_assets_depreciation']+data1_2['intangible_assets_amortization']
data1_2['DEPI']=(data1_2.groupby("code")["折旧摊销"].shift(periods=1)/(data1_2.groupby("code")["fixed_assets"].shift(periods=1)+data1_2.groupby("code")["折旧摊销"].shift(periods=1)))/(data1_2["折旧摊销"]/(data1_2["fixed_assets"]+data1_2["折旧摊销"]))
data1_2['销售管理费用占营收比']=(data1_2['sale_expense']+data1_2['administration_expense'])/data1_2['operating_revenue']
data1_2['SGAI']=data1_2['销售管理费用占营收比']/data1_2.groupby('code')['销售管理费用占营收比'].shift(periods=1)
data1_2['资产负债比']=data1_2['total_liability']/data1_2['total_assets']
data1_2['LVGI']=data1_2['资产负债比']/data1_2.groupby('code')['资产负债比'].shift(periods=1)
data1_2['TATA']=(data1_2['operating_profit']-data1_2['net_operate_cash_flow'])/data1_2['total_assets']
data1_2.fillna(0,inplace=True)
data1_2['M']=-4.84+0.92*data1_2['DSRI']+0.528*data1_2['GMI']+0.404*data1_2['AQI']+0.892*data1_2['SGI']+0.115*data1_2['DEPI']-0.172*data1_2['SGAI']+4.679*data1_2['TATA']-0.327*data1_2['LVGI']
#计算第三个指标：金融化程度
data1_2['RFI']=(data1_2['investment_income']-data1_2['invest_income_associates']+data1_2['other_composite_income'])*2/(data1_2['total_assets']+data1_2.groupby("证券代码")['total_assets'].shift(periods=1))

data1_2=data1_2[["证券代码","statDate","pubDate","intrinsic_value",'M','RFI']]
data1_2.dropna(inplace=True)
del data,data1,data1_1,industry,cost_rate,name,fit1,coef1,fit2,coef2,fit3,coef3
gc.collect()

105

In [5]:
data1_2.head()

,证券代码,statDate,pubDate,intrinsic_value,M,RFI
1,000001.SZ,2011-12-31,2012-03-09,9.368262e+10,3.794860,0.000695
2,000001.SZ,2012-12-31,2013-03-08,1.101960e+11,-3.602250,-0.000216
3,000001.SZ,2013-12-31,2014-03-07,1.361098e+11,-2.739121,-0.000895
4,000001.SZ,2014-12-31,2015-03-13,1.602983e+11,-1.430408,0.001827
5,000001.SZ,2015-12-31,2016-03-10,2.048741e+11,-2.444892,0.001965


In [6]:
gc.collect()
match_date=data1_2[["证券代码","statDate","pubDate"]]
match_date["year"]=match_date["statDate"].apply(lambda x: x.split('-')[0]).astype(int)
match_date["pubDate"]=match_date["pubDate"].apply(lambda x: x.replace("-",""))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
date_data=pd.Series(pd.date_range('1/1/2010','31/12/2019'))
date_data.rename("date")
date_data=pd.DataFrame(date_data.apply(lambda x:x.strftime("%Y-%m-%d").replace('-',"" )))
date_data.columns=["date"]

#tidy市值数据格式
valuation=pd.read_csv("valuation.csv",index_col=0)
valuation["date"]=valuation.index
valuation["date"]=valuation["date"].apply(lambda x:str(x))
valuation=pd.merge(date_data,valuation,on=["date"],how="left")
valuation=valuation.fillna(method='ffill')
valuation.head(20)

,date,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,603988.SH,603989.SH,603990.SH,603991.SH,603992.SH,603993.SH,603996.SH,603997.SH,603998.SH,603999.SH
0,20100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20100104,7362983.5,11654923.0,83976.6875,547685.8125,563987.8125,128181.00000,90446.14062,1182373.625,173627.60938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20100105,7235660.5,11391038.0,83976.6875,566886.8125,547757.9375,132990.09375,90446.14062,1151832.500,170540.23438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20100106,7111443.5,11391038.0,83976.6875,555914.8125,549279.5000,132990.09375,90446.14062,1150741.750,172304.45312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20100107,7033807.5,11303076.0,83976.6875,533056.5000,544714.8125,129105.82031,90446.14062,1172556.875,165541.64062,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20100108,7018280.5,11380043.0,83976.6875,538542.5000,557901.5625,130030.64844,90446.14062,1229275.875,168482.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20100109,7018280.5,11380043.0,83976.6875,538542.5000,557901.5625,130030.64844,90446.14062,1229275.875,168482.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20100110,7018280.5,11380043.0,83976.6875,538542.5000,557901.5625,130030.64844,90446.14062,1229275.875,168482.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
valuation=valuation.melt(id_vars=['date'],var_name='股票代码',value_name='总市值')
valuation["year"]=valuation["date"].apply(lambda x:x[0:4]).astype(int)
valuation["year"]=valuation["year"]-1
valuation=pd.merge(valuation,match_date,left_on=["股票代码","year"],right_on=["证券代码","year"],how="left")
valuation.dropna(inplace=True)
valuation.index=range(len(valuation))
valuation.loc[valuation["date"]<valuation["pubDate"],"year"]=valuation["year"]-1
valuation.head(20)

,date,股票代码,总市值,year,证券代码,statDate,pubDate
0,20120101,000001.SZ,7987303.5,2010,000001.SZ,2011-12-31,20120309
1,20120102,000001.SZ,7987303.5,2010,000001.SZ,2011-12-31,20120309
2,20120103,000001.SZ,7987303.5,2010,000001.SZ,2011-12-31,20120309
3,20120104,000001.SZ,7766999.0,2010,000001.SZ,2011-12-31,20120309
4,20120105,000001.SZ,7884836.5,2010,000001.SZ,2011-12-31,20120309
5,20120106,000001.SZ,7864343.0,2010,000001.SZ,2011-12-31,20120309
6,20120107,000001.SZ,7864343.0,2010,000001.SZ,2011-12-31,20120309
7,20120108,000001.SZ,7864343.0,2010,000001.SZ,2011-12-31,20120309
8,20120109,000001.SZ,8084647.0,2010,000001.SZ,2011-12-31,20120309
9,20120110,000001.SZ,8084647.0,2010,000001.SZ,2011-12-31,20120309


In [18]:
data1_2["year"]=data1_2["statDate"].apply(lambda x: x.split('-')[0]).astype(int)
data=pd.merge(valuation,data1_2,on=["year","证券代码"],how="inner")
del data1_2,valuation,match_date
gc.collect()
data=data[["date","股票代码","总市值","intrinsic_value","M","RFI"]]
data

,date,股票代码,总市值,intrinsic_value,M,RFI
0,20120309,000001.SZ,8.643092e+06,9.368262e+10,3.79486,0.000695
1,20120310,000001.SZ,8.643092e+06,9.368262e+10,3.79486,0.000695
2,20120311,000001.SZ,8.643092e+06,9.368262e+10,3.79486,0.000695
3,20120312,000001.SZ,8.555995e+06,9.368262e+10,3.79486,0.000695
4,20120313,000001.SZ,8.632845e+06,9.368262e+10,3.79486,0.000695
5,20120314,000001.SZ,8.443281e+06,9.368262e+10,3.79486,0.000695
6,20120315,000001.SZ,8.325444e+06,9.368262e+10,3.79486,0.000695
7,20120316,000001.SZ,8.402295e+06,9.368262e+10,3.79486,0.000695
8,20120317,000001.SZ,8.402295e+06,9.368262e+10,3.79486,0.000695
9,20120318,000001.SZ,8.402295e+06,9.368262e+10,3.79486,0.000695


In [19]:
data["ptv"]=data["总市值"]/data["intrinsic_value"]
data.head()

,date,股票代码,总市值,intrinsic_value,M,RFI,ptv
0,20120309,000001.SZ,8643092.0,9.368262e+10,3.79486,0.000695,0.000092
1,20120310,000001.SZ,8643092.0,9.368262e+10,3.79486,0.000695,0.000092
2,20120311,000001.SZ,8643092.0,9.368262e+10,3.79486,0.000695,0.000092
3,20120312,000001.SZ,8555995.0,9.368262e+10,3.79486,0.000695,0.000091
4,20120313,000001.SZ,8632845.0,9.368262e+10,3.79486,0.000695,0.000092
